In [ ]:
# cleaning...%load_ext autoreload
# cleaning...%autoreload 2

In [ ]:
from __init__ import *

In [ ]:
gps_map = kraft.read_gps_map(PATH["gps_map.pickle.gz"])

In [ ]:
feature_data_dicts = SETTING["feature_data"]

feature_data_dicts = {
    data_name: data_dict
    for data_name, data_dict in feature_data_dicts.items()
    if "indices_to_peek" in data_dict
}

for data_name, data_dict in feature_data_dicts.items():

    print(data_name)

    data_dict["dataframe"] = pd.read_csv(data_dict["file_path"], sep="\t", index_col=0)

In [ ]:
sample_data_dicts = SETTING["sample_data"]

sample_data_dicts = {
    data_name: data_dict
    for data_name, data_dict in sample_data_dicts.items()
    if "indices_to_peek" in data_dict
}

for data_name, data_dict in sample_data_dicts.items():

    print(data_name)

    data_dict["dataframe"] = pd.read_csv(data_dict["file_path"], sep="\t", index_col=0)

In [ ]:
for w_or_h, data_dicts in (("w", feature_data_dicts), ("h", sample_data_dicts)):

    for factor_name, data_name_indices in SETTING["{}_summary".format(w_or_h)].items():

        for data_name, indices in data_name_indices.items():

            if data_dicts[data_name]["indices_to_peek"] == "all":

                continue

            data_dicts[data_name]["indices_to_peek"] += indices

In [ ]:
for element_type, w_or_h, elements, element_label, colorscale, data_dicts in (
    (
        "feature",
        "w",
        gps_map.w_elements,
        gps_map.w_element_label,
        gps_map.w_label_colorscale,
        feature_data_dicts,
    ),
    (
        "sample",
        "h",
        gps_map.h_elements,
        gps_map.h_element_label,
        gps_map.h_label_colorscale,
        sample_data_dicts,
    ),
):

    for data_name, data_dict in data_dicts.items():

        indices_to_peek = data_dict["indices_to_peek"]

        if indices_to_peek == "all":

            indices_to_peek = data_dict["dataframe"].index

        element_marker_size = SETTING["gps_map_{}_element_marker_size".format(w_or_h)]

        for index, element_value in (
            data_dict["dataframe"]
            .reindex(index=indices_to_peek)
            .dropna(how="all")
            .iterrows()
        ):

            html_file_name = kraft.normalize_file_name(
                "{}.{}.html".format(data_name, index)
            )

            print(html_file_name)

            gps_map.plot(
                w_or_h,
                element_value=element_value,
                element_value_data_type=data_dict["data_type"],
                element_marker_size=element_marker_size,
                html_file_path=os.path.join(
                    PATH["{}|gps_map/".format(w_or_h)], html_file_name
                ),
            )

            if element_label is None:

                continue

            label_element_value = element_value.groupby(by=element_label)

            layout = {
                "title": {"text": data_name},
                "xaxis": {"dtick": 1, "title": {"text": "Cluster"}},
                "yaxis": {"title": {"text": index}},
            }

            html_file_path = os.path.join(
                PATH["{}|hcc|comparison/".format(w_or_h)], html_file_name
            )

            label_color = {
                label: kraft.get_colorscale_color(
                    colorscale, label, len(label_element_value)
                )
                for label in label_element_value.groups.keys()
            }

            if data_dict["data_type"] == "binary":

                kraft.plot_plotly_figure(
                    {
                        "layout": layout,
                        "data": [
                            {
                                "type": "bar",
                                "name": label,
                                "x": (label,),
                                "y": (element_value_.sum() / element_value_.size,),
                                "marker": {"color": label_color[label]},
                            }
                            for label, element_value_ in label_element_value
                        ],
                    },
                    html_file_path,
                )

            else:

                kraft.plot_plotly_figure(
                    {
                        "layout": layout,
                        "data": [
                            {
                                "type": "violin",
                                "name": label,
                                "y": element_value_,
                                "marker": {"color": label_color[label]},
                                "scalemode": "count",
                                "meanline": {"visible": True},
                                "points": "all",
                            }
                            for label, element_value_ in label_element_value
                        ],
                    },
                    html_file_path,
                )